In [1]:
import os
import random
import shutil
import warnings

import matplotlib
matplotlib.use("TKAgg")

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from imutils import paths
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [ ]:
from VGG16 import VGG16
from mobileNet import mobileNet
from VGG19 import VGG19
from Resnet50 import Resnet50
from inception import inception
from utility import train, evaluate

In [ ]:
plt.style.use('fivethirtyeight')
plt.rcParams['figure.figsize'] = [10, 5]
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
dataset = 'COVID-Datasets'
imagePaths = sorted(list(paths.list_images(dataset)))
random.seed(42)
random.shuffle(imagePaths)

In [ ]:
imagesData = []
labels = []

codeLabel = {1: 'COVID', 0: 'Normal'}
labelCode = {}
for k, v in codeLabel.items():
    labelCode[v] = k

if not os.path.isdir("./models"):
    print("Creating directory to store models")
    os.mkdir("./models")

temp = {}
np.save('./models/labelName.npy', codeLabel)

In [ ]:
for i, imagePath in enumerate(imagePaths):
    try:
        image = cv2.imread(imagePath, 1)
        image = image[..., ::-1]
        image = cv2.resize(image, (224, 224))
        image = (image / 255.).astype(np.float32)

        label = imagePath.split(os.path.sep)[-2]
        if label not in temp:
            temp[label] = 1
        else:
            temp[label] += 1
        if temp[label] <= 360:
            imagesData.append(image)
            labels.append(labelCode[label])

    except (FileNotFoundError, Exception) as e:
        print(imagePath)
        pass

labels = np.array(labels)
print("Output data shape : ", labels.shape)

In [ ]:
# splitting dataset
trainX, testX, trainY, testY = train_test_split(imagesData, labels, test_size=0.2)

print(np.array(trainX).shape)
print(np.array(trainY).shape)

In [ ]:
# Create the directory structure
base_dir = 'dataDir'
if not os.path.isdir(base_dir):
    print("Creating base_dir")
    os.mkdir(base_dir)

In [ ]:
# train_dir
train_dir = os.path.join(base_dir, 'train_dir')
if not os.path.isdir(train_dir):
    print("Creating train_dir")
    os.mkdir(train_dir)

In [ ]:
# val_dir
val_dir = os.path.join(base_dir, 'val_dir')
if not os.path.isdir(val_dir):
    print("Creating val_dir")
    os.mkdir(val_dir)

In [ ]:
idx = 1
for image, label in zip(trainX, trainY):
    label_dir = os.path.join(train_dir, str(label))
    if not os.path.isdir(label_dir):
        os.mkdir(label_dir)
    dst = os.path.join(label_dir, "label_{}.png".format(idx))
    image = cv2.convertScaleAbs(image, alpha=(255.0))
    cv2.imwrite(dst, image)
    idx += 1

idx = 1
for image, label in zip(testX, testY):
    label_dir = os.path.join(val_dir, str(label))
    if not os.path.isdir(label_dir):
        os.mkdir(label_dir)
    dst = os.path.join(label_dir, "label_{}.png".format(idx))
    image = cv2.convertScaleAbs(image, alpha=(255.0))
    cv2.imwrite(dst, image)
    idx += 1

In [ ]:
# Set Up the Generators
train_path = 'dataDir/train_dir'
val_path = 'dataDir/val_dir'

datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_gen = datagen.flow_from_directory(train_path, target_size=(224, 224), batch_size=64)

val_gen = datagen.flow_from_directory(val_path, target_size=(224, 224), batch_size=64)

test_gen = datagen.flow_from_directory(val_path, target_size=(224, 224), batch_size=1, shuffle=False)

In [ ]:
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ mobileNet model ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# defining mobileNet model
print("Defining mobileNet model")
model = mobileNet()
model.summary()

print()
print("Total number of layers in Model :", len(model.layers))

x = model.layers[-6].output
x = Dropout(0.25)(x)
predictions = Dense(len(set(labels)), activation='softmax')(x)
models = Model(inputs=model.input, outputs=predictions)

models.summary()

for layer in models.layers[:-23]:
    layer.trainable = False

models.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

# training the model MobileNet
print("Training the model MobileNet")
train("MobileNet", models, train_gen, val_gen)

print("Evaluating the model MobileNet")
evaluate("MobileNet", test_gen)